In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import LambdaLR
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier


In [ ]:
path_to_folder = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/Outputs_project_data_preprocessing'
path = "C:/Users/Admin/Thesis/data/ankiety"

In [ ]:
import os
import json
from tqdm import tqdm
list_patient = []
list_id_patient = os.listdir(path)
list_of_data_for_all_patients = []

for iter in tqdm(list_id_patient):
#     print('Patient: ', patient )
    lista_id_iter = os.listdir(f'{path}/{iter}')
    for patient in tqdm(lista_id_iter):
    #     print('Patient: ', patient )
        lista_id_emocion_patient = os.listdir(f'{path}/{iter}/{patient}/assessments/emotion')
        for emotion in lista_id_emocion_patient:
    #         print("Emotion: ", emotion)
            f = open(f'{path}/{iter}/{patient}/assessments/emotion/{emotion}', encoding="utf8")
            data = json.load(f)
            list_of_data_for_all_patients.append(data)
            list_patient.append(patient)

In [ ]:
list_id_patient

In [ ]:
import pandas as pd
from glob import glob

df = pd.DataFrame()
for iter in tqdm(list_id_patient):
    lista_id_iter = os.listdir(f'{path}/{iter}')
    for patient in tqdm(lista_id_iter):
        patient_data = pd.concat([pd.read_json(f_name, lines=True) for f_name in glob(f'{path}/{iter}/{patient}/assessments/emotion/*.json')])
        patient_data['patient_id'] = patient
        df = pd.concat([df, patient_data])

In [ ]:
emotions_df = df.copy().reset_index(drop=True)
emotions_df

In [ ]:
emotions_df['emotionTimestamp'][25192]


In [ ]:
emotions_df['emotionTimestamp'] = emotions_df['emotionTimestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f') if '.' in x else datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

emotions_df['date'] = pd.to_datetime(emotions_df['emotionTimestamp'])
emotions_df['hour'] = emotions_df['date'].dt.hour
emotions_df['dateOfSleep'] = pd.to_datetime(emotions_df['date']).dt.floor('D')

emotions_df['dateOfSleep'] = emotions_df['dateOfSleep'] + pd.Timedelta(days=1)

In [ ]:
emotions_df

In [ ]:
result_df = emotions_df.copy() 

contains_dict = result_df.map(lambda x: isinstance(x, dict)).any()
print("Columns containing dictionaries:")
print(contains_dict)

columns_with_dicts = contains_dict[contains_dict].index.tolist()
print("Columns containing dictionaries:")
print(columns_with_dicts)


list_of_columns_with_dict = columns_with_dicts
for column in list_of_columns_with_dict:
    # Expanding predictions
#     display(result_df.head(3))
    result_df[column].fillna({}, inplace=True)
    expanded_df = pd.json_normalize(result_df[column])
    result_df.reset_index(drop=True, inplace=True)
    expanded_df.reset_index(drop=True, inplace=True)
    result_df = pd.concat([result_df, expanded_df], axis=1)
    result_df = result_df.drop(column, axis=1)

In [ ]:
result_df.columns

In [ ]:
emotions_df = result_df.copy()

In [ ]:
emotions_df

In [ ]:
emotions_df['intenseEmotion'].unique()

## join features in one column

In [ ]:
unique_values = {np.nan: 0, 'DONT_KNOW': 1, 'NO': 2, 'YES': 3}
emotions_df.replace({'intenseEmotion': unique_values}, inplace=True)

In [ ]:
emotions_df['intenseEmotion'].unique()

In [ ]:
emotions_df['features'] = emotions_df[['intenseEmotion', 'valence', 'arousal']].values.tolist()

# Group by date and id, and aggregate features into a list
# grouped = emotions_df.groupby([ 'patient_id'])['features'].agg(list).reset_index()

# Drop individual feature columns if needed
# grouped.drop(['intenseEmotion', 'valence', 'arousal', 'hour'], axis=1, inplace=True)
# grouped_df = emotions_df.groupby('dateOfSleep').agg({ 'features': lambda x: list(x)}).reset_index()

# grouped_df['dateOfSleep']

In [ ]:
emotions_df

In [ ]:
grouped_df = emotions_df.groupby(['patient_id', 'dateOfSleep'])[['intenseEmotion', 'valence', 'arousal']].apply(lambda x: x.values.tolist()).reset_index()

In [ ]:
grouped_df[0][0][0][0]

In [ ]:
len(list(grouped_df['patient_id'].unique()))

In [ ]:
grouped_df.rename(columns={0: 'features'}, inplace=True)

In [ ]:
grouped_df

In [ ]:
grouped_df.to_pickle(f'{path_to_folder}/6_emotions_aggregates/features_emotion.pkl')